### 05 AI Agent Workflow + RAG（検索拡張生成）
gpt-oss:20bを使用する構成のため、**Colab GPU は L4 を使用すること。**
- 必要なライブラリをインストール
- Google Colab に Ollama をセットアップ
  - LLM モデルは gpt-oss:20b を使用（Ollama）
  - Embedding モデルは ruri-v3-310m を使用（Sentence Transformers）
  - Reranker モデルは cl-nagoya/ruri-v3-reranker-310m を使用（Sentence Transformers）
- JAXA（宇宙航空研究開発機構）のリポジトリからデータをダウンロードして読み込み
> [井澤克彦, 市川信一郎, 高速回転ホイール: 高速回転ホイール開発を通しての知見, 宇宙航空研究開発機構研究開発報告, 2008](https://jaxa.repo.nii.ac.jp/records/2149)
- データの前処理
  - markdown に変換（MarkItDown を使用）
  - Unicode正規化 (NFKC), 1文字行ブロックの除去, 空行圧縮
  - チャンク分割
    - LangChain の SpacyTextSplitter を使用
    - spaCy の日本語モデルは、ja_ginza を使用
- ベクトルデータベースの構築（ChromaDB, インメモリ）
- 検索機能の実装
  - キーワード検索 @ BM25（spaCyで形態素解析の前処理が必要）
  - Embedding model によるセマンティック検索
  - ハイブリッド検索
  - Reranker による再順位付け
  - 検索機能をLLM の tool として定義
- LangGraph による Workflow の実装
  1. ユーザの質問を入力。
  2. ユーザの質問に回答するためのタスク分割, 作成。
  3. tool による検索。
  4. tool による検索を終えて回答作成に進むか判断。再調査なら 3 に戻る。
  5. ユーザへの回答の作成と提示。
- 動作確認

**必要なライブラリをインストール**

In [1]:
# Google Colab に必要なライブラリをインストールする。
# 1行にまとめることで pip が全パッケージの依存関係を一括解決する。
# NOTE: Colab では uv ではなく pip を使う。uv は依存解決の過程で
#       numpy 等をアップグレードし、プリインストール済みの scipy 等を壊すため。
# NOTE: langchain 関連は 1.x 系に明示的に指定する。
#       Colab プリインストールの 0.3.x が残ると langchain-mcp-adapters が動作しない。
%pip install -U ollama langchain-ollama \
     "langchain>=1.2.8" "langchain-core>=1.2.8" \
     "langgraph>=1.0.7" \
     "markitdown[all]" chromadb \
     "langchain-text-splitters>=0.3" \
     spacy ginza ja-ginza \
     rank-bm25 sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.1/500.1 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 121.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 39.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 140.3 MB/s eta 0:00:0000:010:01
   ━━━━━━

**Google Colab に Ollama をセットアップ**

In [2]:
# Ollama のインストール・起動・モデルのダウンロード
# 詳細は 01_connect_oss_llm.ipynb を参照
import subprocess
import time
import ollama  # type: ignore

!apt-get install -y -qq zstd
!curl -fsSL https://ollama.com/install.sh | sh

process = subprocess.Popen(
    ["ollama", "serve"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)
time.sleep(5)


def ollama_pull(model: str) -> None:
    """Ollama モデルをダウンロードし、進捗をインライン表示する。"""
    for progress in ollama.pull(model, stream=True):
        status = progress.get("status", "")
        total = progress.get("total") or 0
        completed = progress.get("completed") or 0
        if total:
            line = f"{status}: {completed / total:.0%}"
        else:
            line = status
        print(f"\r{line:<60}", end="", flush=True)
    print(f"\n{model}: Done!")


model_name = "gpt-oss:20b"
ollama_pull(model_name)
!ollama show {model_name}

Selecting previously unselected package zstd.
(Reading database ... 121852 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
success                                                     
gpt-oss:20b: Done!
  Model
    architecture        gptoss    
    parameters          20.9B     
    context length      131072    
    embedding length    2880      
    quantization        MXFP4     

  Ca

**ChatOllama で LLM に接続**

In [3]:
# ChatOllama で LLM に接続する。
from langchain_ollama import ChatOllama  # type: ignore

llm = ChatOllama(
    model="gpt-oss:20b",
    num_ctx=16384,
    num_predict=-1,
    temperature=0.8,
    top_k=40,
    top_p=0.9,
    repeat_penalty=1.1,
    reasoning=None,
)

**Embedding モデル（ruri-v3-310m）と Reranker モデルのセットアップ**

In [4]:
# Embedding: ruri-v3-310m (Sentence Transformers 経由)
from langchain_core.embeddings import Embeddings  # type: ignore
from sentence_transformers import SentenceTransformer, CrossEncoder  # type: ignore


class RuriEmbeddings(Embeddings):
    """ruri-v3 を LangChain の Embeddings インターフェースでラップする。"""

    def __init__(self, model_name: str = "cl-nagoya/ruri-v3-310m") -> None:
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        prefixed = [f"検索文書: {t}" for t in texts]
        return self.model.encode(prefixed).tolist()

    def embed_query(self, text: str) -> list[float]:
        return self.model.encode(f"検索クエリ: {text}").tolist()


embeddings = RuriEmbeddings()
test_vec = embeddings.embed_query("テスト文です")
print(f"Embedding dim: {len(test_vec)}")

# Reranker: cl-nagoya/ruri-v3-reranker-310m
reranker = CrossEncoder("cl-nagoya/ruri-v3-reranker-310m")
print("Reranker model loaded.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/1.83M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/968 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Reranker model loaded.


**JAXA リポジトリからデータをダウンロード → 前処理 → チャンク分割**

In [5]:
# JAXA リポジトリから PDF をダウンロードし、MarkItDown で markdown に変換する。
import urllib.request
import unicodedata
import re
from pathlib import Path
from markitdown import MarkItDown  # type: ignore

pdf_url = "https://jaxa.repo.nii.ac.jp/record/2149/files/63826000.pdf"
pdf_path = Path("高速回転ホイール.pdf")

if not pdf_path.exists():
    urllib.request.urlretrieve(pdf_url, pdf_path)
    print(f"ダウンロード完了: {pdf_path}")

md = MarkItDown()
result = md.convert(str(pdf_path))
raw_text = result.text_content
print(f"変換後の文字数: {len(raw_text)}")

# Unicode 正規化 (NFKC)
text = unicodedata.normalize("NFKC", raw_text)


# PDF 抽出テキストの汎用クリーニング
def clean_pdf_text(text: str) -> str:
    """1文字行ブロックの除去 + 空行圧縮。"""
    text = re.sub(
        r"(^[^\S\n]*\S[^\S\n]*$\n?){3,}",
        "\n",
        text,
        flags=re.MULTILINE,
    )
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


text = clean_pdf_text(text)
print(f"クリーニング後の文字数: {len(text)}")

ダウンロード完了: 高速回転ホイール.pdf
変換後の文字数: 86888
クリーニング後の文字数: 79515


In [ ]:
# SpacyTextSplitter でチャンク分割する。
# Sudachi の入力上限 (49,149 bytes) を超えないように事前分割してから渡す。
from langchain_text_splitters import SpacyTextSplitter  # type: ignore

CHUNK_SIZE = 500
CHUNK_OVERLAP = 100
BLOCK_MAX_BYTES = 40_000
BLOCK_OVERLAP_CHARS = CHUNK_SIZE


def split_into_safe_blocks(
    text: str,
    max_bytes: int = BLOCK_MAX_BYTES,
    overlap_chars: int = BLOCK_OVERLAP_CHARS,
) -> list[str]:
    """テキストを段落区切りで max_bytes 以下のブロックに分割する。"""
    paragraphs = text.split("\n\n")
    blocks: list[str] = []
    current = ""
    for para in paragraphs:
        candidate = current + "\n\n" + para if current else para
        if len(candidate.encode("utf-8")) > max_bytes and current:
            blocks.append(current)
            current = current[-overlap_chars:] + "\n\n" + para
        else:
            current = candidate
    if current:
        blocks.append(current)
    return blocks


text_splitter = SpacyTextSplitter(
    separator="\n\n",
    pipeline="ja_ginza",
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
)

blocks = split_into_safe_blocks(text)
chunks: list[str] = []
for block in blocks:
    chunks.extend(text_splitter.split_text(block))

print(f"チャンク数: {len(chunks)}")

チャンク数: 76


**ベクトルデータベースの構築（ChromaDB, インメモリ）**

In [7]:
# ChromaDB にチャンクを格納する（インメモリ）。
import chromadb  # type: ignore

chroma_client = chromadb.Client()

collection = chroma_client.create_collection(
    name="jaxa_wheel",
    metadata={"hnsw:space": "cosine"},
)

chunk_embeddings = embeddings.embed_documents(chunks)

collection.add(
    ids=[f"chunk_{i}" for i in range(len(chunks))],
    documents=chunks,
    embeddings=chunk_embeddings,
)

print(f"ChromaDB に {collection.count()} 件のチャンクを格納しました。")

ChromaDB に 76 件のチャンクを格納しました。


**検索機能の実装（BM25 + セマンティック + ハイブリッド + Reranker）と tool 定義**

In [8]:
# BM25 用の前処理: spaCy (ja_ginza) で形態素解析してトークン化する。
import numpy as np  # type: ignore
import spacy  # type: ignore
from rank_bm25 import BM25Okapi  # type: ignore

nlp = spacy.load("ja_ginza", disable=["parser", "ner"])


def tokenize(text: str) -> list[str]:
    """spaCy で形態素解析し、BM25 用のトークンリストを返す。"""
    doc = nlp(text)
    tokens = []
    include_pos = {"NOUN", "VERB", "ADJ", "PROPN", "NUM"}
    for token in doc:
        if token.pos_ not in include_pos:
            continue
        if token.is_stop:
            continue
        lemma = token.lemma_
        if len(lemma) == 1 and re.match(r"[ぁ-ん\u30fc!-/:-@\[-`{-~]", lemma):
            continue
        tokens.append(lemma)
    return tokens


tokenized_chunks = [tokenize(chunk) for chunk in chunks]
bm25 = BM25Okapi(tokenized_chunks)
print(f"BM25 インデックス構築完了: {len(tokenized_chunks)} 件")

BM25 インデックス構築完了: 76 件


In [ ]:
# 検索関数の定義（BM25, セマンティック, ハイブリッド, Reranker）
RETRIEVAL_TOP_K = 20
RERANK_TOP_K = 5
BM25_WEIGHT = 0.3


def search_bm25(query: str, top_k: int = RETRIEVAL_TOP_K) -> list[dict]:
    """BM25 によるキーワード検索を行う。"""
    tokenized_query = tokenize(query)
    scores = bm25.get_scores(tokenized_query)
    top_indices = np.argsort(scores)[::-1][:top_k]
    return [
        {
            "rank": rank + 1,
            "chunk_id": int(idx),
            "score": float(scores[idx]),
            "text": chunks[idx],
        }
        for rank, idx in enumerate(top_indices)
        if scores[idx] > 0
    ]


def search_semantic(query: str, top_k: int = RETRIEVAL_TOP_K) -> list[dict]:
    """Embedding model によるセマンティック検索を行う。"""
    query_embedding = embeddings.embed_query(query)
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    return [
        {
            "rank": rank + 1,
            "chunk_id": int(doc_id.split("_")[1]),
            "score": 1.0 - dist,
            "text": doc,
        }
        for rank, (doc_id, doc, dist) in enumerate(
            zip(results["ids"][0], results["documents"][0], results["distances"][0])
        )
    ]


def search_hybrid(
    query: str, top_k: int = RETRIEVAL_TOP_K, bm25_weight: float = BM25_WEIGHT
) -> list[dict]:
    """BM25 とセマンティック検索の RRF ハイブリッド検索を行う。"""
    k = 60
    bm25_results = search_bm25(query, top_k=top_k)
    semantic_results = search_semantic(query, top_k=top_k)

    scores: dict[int, float] = {}
    texts: dict[int, str] = {}

    for r in bm25_results:
        cid = r["chunk_id"]
        scores[cid] = scores.get(cid, 0) + bm25_weight / (k + r["rank"])
        texts[cid] = r["text"]

    semantic_weight = 1.0 - bm25_weight
    for r in semantic_results:
        cid = r["chunk_id"]
        scores[cid] = scores.get(cid, 0) + semantic_weight / (k + r["rank"])
        texts[cid] = r["text"]

    sorted_ids = sorted(scores, key=lambda cid: scores[cid], reverse=True)[:top_k]
    return [
        {"rank": rank + 1, "chunk_id": cid, "score": scores[cid], "text": texts[cid]}
        for rank, cid in enumerate(sorted_ids)
    ]


def rerank(query: str, results: list[dict], top_k: int = RERANK_TOP_K) -> list[dict]:
    """Reranker (CrossEncoder) で検索結果を再順位付けする。"""
    if not results:
        return []
    pairs = [(query, r["text"]) for r in results]
    scores = reranker.predict(pairs)
    ranked_indices = np.argsort(scores)[::-1][:top_k]
    return [
        {
            "rank": rank + 1,
            "chunk_id": results[idx]["chunk_id"],
            "score": float(scores[idx]),
            "text": results[idx]["text"],
        }
        for rank, idx in enumerate(ranked_indices)
    ]


print("検索関数を定義しました: search_bm25, search_semantic, search_hybrid, rerank")

In [ ]:
# 検索機能を LLM の tool として定義する。
from langchain_core.tools import tool  # type: ignore

MAX_RETURN_CHARS = 8000


@tool
def search_document(query: str) -> str:
    """外部ナレッジベースから、クエリに関連する情報を検索・取得します。
    ユーザーの質問に対し、具体的な事実、データ、あるいは詳細な文脈が必要な場合、
    自身の知識だけで回答せずに必ずこのツールを使用してください。

    Args:
        query: 検索したい内容を表す、具体的かつ完全な文章（日本語）。
    """
    try:
        hybrid_results = search_hybrid(query)
        reranked = rerank(query, hybrid_results)
    except Exception as e:
        return f"検索中にエラーが発生しました: {e}"

    if not reranked:
        return "検索結果が見つかりませんでした。"

    passages = []
    total_chars = 0
    for r in reranked:
        passage = f"[チャンク {r['chunk_id']}] (スコア: {r['score']:.4f})\n{r['text']}"
        total_chars += len(passage)
        if total_chars > MAX_RETURN_CHARS:
            break
        passages.append(passage)
    return "\n\n---\n\n".join(passages)


search_tool = search_document
print(f"RAG Tool: {search_tool.name}")

**LangGraph による Workflow の実装**

03_02 の Workflow を RAG 検索用に適応する。web_search ノードを doc_search ノードに置き換え、
MCP サーバの代わりに `search_document` ツールで検索する。

**Workflow の流れ**
1. **task_planning**: ユーザの質問を受け取り、回答に必要なサブタスク（目的＋検索クエリ）を構造化して作成する。
2. **doc_search**: 各サブタスクの検索クエリを `search_document` ツールで実行し、目的と紐付けた検索結果を蓄積する。
3. **judge**: サブタスクの目的ごとに、検索結果が十分かを LLM が判断する。不足なら追加サブタスクを生成して doc_search に戻る。
4. **generate_answer**: 目的ごとに整理された検索結果をもとに、ユーザの質問に対する最終回答を生成する。

In [ ]:
# Workflow の状態定義・共通ユーティリティ・システムプロンプト
import json
from typing import TypedDict
from langchain_core.messages import HumanMessage, SystemMessage  # type: ignore
from langgraph.graph import StateGraph, START, END  # type: ignore
from IPython.display import Image, display

# --- グローバル設定 ---
MAX_LOOP_COUNT = 2  # judge → doc_search 再調査ループの上限回数


# --- Workflow の状態 ---
class WorkflowState(TypedDict):
    question: str
    subtasks: list[dict]  # [{"purpose": str, "queries": [str]}]
    search_results: list[str]  # 目的と紐付けた検索結果
    answer: str
    loop_count: int


# --- 共通ユーティリティ: LLM 出力から JSON を抽出 ---
def extract_json_text(raw: str) -> str:
    """LLM の出力から JSON 文字列を抽出する。"""
    text = raw.strip()

    code_block_match = re.search(r"```(?:json)?\s*(.*?)```", text, re.DOTALL)
    if code_block_match:
        text = code_block_match.group(1).strip()

    try:
        json.loads(text)
        return text
    except json.JSONDecodeError:
        pass

    match_start = re.search(r"[\{\[]", text)
    if not match_start:
        return text

    first_brace_index = match_start.start()
    start_char = text[first_brace_index]
    end_char = "]" if start_char == "[" else "}"

    last_brace_index = text.rfind(end_char)
    if last_brace_index > first_brace_index:
        return text[first_brace_index : last_brace_index + 1]

    return text


# --- 各ノードのシステムプロンプト ---
SYSTEM_PROMPT_TASK_PLANNING = """\
あなたはリサーチプランナーです。
ユーザの質問に回答するために、ナレッジベース（技術文書）を検索するためのサブタスクを作成してください。

出力は以下の JSON 配列のみとし、他のテキストは一切含めないでください。
サブタスクは最大3個までとしてください。

出力形式:
[
  {"purpose": "このサブタスクで明らかにしたいこと",
   "queries": ["検索クエリ1", "検索クエリ2"]},
  ...
]

purpose は判定ステップで「この目的に十分な情報が得られたか」を評価する基準になります。
具体的かつ明確に書いてください。
検索クエリは、技術文書から関連情報を検索するための日本語の具体的なフレーズにしてください。
"""

SYSTEM_PROMPT_JUDGE = """\
あなたはリサーチの品質を判定する審査員です。
ユーザの質問と検索結果を見て、回答に十分な情報があるか判断してください。
検索結果には【目的: ...】タグが付いています。
各目的について十分な情報が得られているかを確認してください。

十分な場合:
{"sufficient": true, "reason": "判断理由を日本語で1文で"}

不足の場合（不足している目的について追加サブタスクを生成）:
{"sufficient": false, "reason": "何が不足しているかを日本語で1文で",
 "additional_subtasks": [
    {"purpose": "追加で明らかにしたいこと",
     "queries": ["追加クエリ1"]}
  ]
}

JSON のみ出力し、他のテキストは含めないでください。
"""

SYSTEM_PROMPT_GENERATE_ANSWER = """\
あなたはリサーチ結果をもとに回答するAIアシスタントです。
検索結果を参考に、ユーザの質問に日本語で丁寧に回答してください。
回答は必ず検索結果に基づいて作成し、検索結果に含まれない情報は含めないでください。
回答の最後に、以下の形式で結論をまとめてください。

# 結論
- ユーザの質問: （質問内容）
- 回答: （簡潔な回答）
"""

print("Workflow の状態定義・ユーティリティ・システムプロンプトを定義しました。")

In [ ]:
# Workflow ノードの定義

# ノード 1: task_planning（タスク分割）
async def task_planning(state: WorkflowState) -> dict:
    """ユーザの質問を分析し、サブタスク（目的＋検索クエリ）を作成する。"""
    question = state["question"]

    response = await llm.ainvoke(
        [
            SystemMessage(content=SYSTEM_PROMPT_TASK_PLANNING),
            HumanMessage(content=question),
        ]
    )

    text = extract_json_text(response.content)

    try:
        subtasks = json.loads(text)
    except json.JSONDecodeError:
        print(f"[task_planning] JSON パース失敗 → フォールバック: {text[:100]}")
        subtasks = [{"purpose": "基本調査", "queries": [question]}]

    print(f"[task_planning] サブタスク数: {len(subtasks)}")
    for i, st in enumerate(subtasks):
        print(f"  {i + 1}. 目的: {st['purpose']}")
        print(f"     クエリ: {st['queries']}")
    return {"subtasks": subtasks, "search_results": [], "loop_count": 0}


# ノード 2: doc_search（ドキュメント検索）
async def doc_search(state: WorkflowState) -> dict:
    """各サブタスクの検索クエリを search_document ツールで実行し、結果を蓄積する。"""
    subtasks = state["subtasks"]
    results = list(state.get("search_results") or [])

    for st in subtasks:
        purpose = st["purpose"]
        print(f"[doc_search] 目的: {purpose}")
        for query in st["queries"]:
            print(f"  検索中: {query}")
            try:
                result = search_tool.invoke({"query": query})
            except Exception as e:
                print(f"  [ERROR] クエリ失敗: {query} → {e}")
                continue
            if not result or result == "検索結果が見つかりませんでした。":
                print(f"  [SKIP] 検索結果なし: {query}")
                continue
            results.append(f"【目的: {purpose}】\n【クエリ: {query}】\n{result}")

    return {"search_results": results, "subtasks": []}


# ノード 3: judge（判定）
async def judge(state: WorkflowState) -> dict:
    """検索結果が十分かを判断し、不足なら追加サブタスクを生成する。"""
    question = state["question"]
    results = state["search_results"]
    loop_count = state.get("loop_count", 0)

    if loop_count >= MAX_LOOP_COUNT:
        print("[judge] ループ上限に到達 → 回答作成へ")
        return {"subtasks": [], "loop_count": loop_count}

    results_text = "\n\n".join(results)

    response = await llm.ainvoke(
        [
            SystemMessage(content=SYSTEM_PROMPT_JUDGE),
            HumanMessage(content=f"質問: {question}\n\n検索結果:\n{results_text}"),
        ]
    )

    text = extract_json_text(response.content)

    try:
        judgment = json.loads(text)
    except json.JSONDecodeError:
        print(f"[judge] JSON パース失敗 → 回答作成へ: {text[:100]}")
        return {"subtasks": [], "loop_count": loop_count + 1}

    reason = judgment.get("reason", "")

    if judgment.get("sufficient", True):
        print(f"[judge] 情報十分 → 回答作成へ（理由: {reason}）")
        return {"subtasks": [], "loop_count": loop_count + 1}
    else:
        additional = judgment.get("additional_subtasks", [])
        print(f"[judge] 情報不足（理由: {reason}）→ 追加サブタスク:")
        for i, st in enumerate(additional):
            print(f"  {i + 1}. 目的: {st.get('purpose', '?')}")
            print(f"     クエリ: {st.get('queries', [])}")
        return {"subtasks": additional, "loop_count": loop_count + 1}


# ルーター: judge の結果で分岐
def should_continue_search(state: WorkflowState) -> str:
    """追加サブタスクがあれば doc_search に戻り、なければ回答生成へ。"""
    if state.get("subtasks"):
        return "doc_search"
    return "generate_answer"


# ノード 4: generate_answer（回答生成）
async def generate_answer(state: WorkflowState) -> dict:
    """蓄積した検索結果をもとに最終回答を生成する。"""
    question = state["question"]
    results_text = "\n\n".join(state["search_results"])

    response = await llm.ainvoke(
        [
            SystemMessage(content=SYSTEM_PROMPT_GENERATE_ANSWER),
            HumanMessage(content=f"質問: {question}\n\n検索結果:\n{results_text}"),
        ]
    )

    answer = response.content or ""

    if not answer:
        print("[generate_answer] WARNING: response.content が空です")
        print(f"  response type: {type(response)}")
        print(f"  response repr: {repr(response)[:500]}")

    print("[generate_answer] 回答生成完了")
    return {"answer": answer}


print(
    "Workflow ノードを定義しました: task_planning, doc_search, judge, generate_answer"
)

In [ ]:
# Workflow グラフの構築とコンパイル
workflow = StateGraph(WorkflowState)

# ノードの登録
workflow.add_node("task_planning", task_planning)
workflow.add_node("doc_search", doc_search)
workflow.add_node("judge", judge)
workflow.add_node("generate_answer", generate_answer)

# エッジの定義
workflow.add_edge(START, "task_planning")
workflow.add_edge("task_planning", "doc_search")
workflow.add_edge("doc_search", "judge")

# 条件分岐: judge → doc_search（再調査） or generate_answer（回答生成）
workflow.add_conditional_edges(
    "judge",
    should_continue_search,
    {
        "doc_search": "doc_search",
        "generate_answer": "generate_answer",
    },
)
workflow.add_edge("generate_answer", END)

# コンパイル
app = workflow.compile()

# グラフの可視化
display(Image(app.get_graph().draw_mermaid_png()))

**動作確認**

In [ ]:
# Workflow エージェントの動作確認
import io
import sys
from IPython.display import Markdown, HTML, display

# 中間ログをキャプチャしつつ、リアルタイムでもセルに出力する
log_buffer = io.StringIO()


class TeeStream:
    """stdout への出力を画面表示しつつバッファにも記録する。"""

    def __init__(self, original, buffer):
        self.original = original
        self.buffer = buffer

    def write(self, text):
        self.original.write(text)
        self.buffer.write(text)

    def flush(self):
        self.original.flush()


_original_stdout = sys.stdout
sys.stdout = TeeStream(_original_stdout, log_buffer)
try:
    result = await app.ainvoke(
        {"question": "高速回転ホイールの寿命試験ではどのような結果が得られましたか？"}
    )
finally:
    sys.stdout = _original_stdout

# 中間ログを HTML で全文表示（Colab のセル出力トランケートを回避）
log_text = log_buffer.getvalue()
print("\n--- 以下は HTML による全文ログ（トランケート回避） ---")
display(HTML(f"<pre style='white-space:pre-wrap'>{log_text}</pre>"))

# 最終回答の表示
print("=== Workflow エージェントの実行結果 ===\n")
answer = result.get("answer", "")
if answer:
    display(Markdown(answer))
else:
    print("[WARNING] 回答が空です。result keys:", list(result.keys()))
    print("search_results 件数:", len(result.get("search_results", [])))
    print("loop_count:", result.get("loop_count"))